In [1]:
!head -5 ./data/totalScor_1121.csv

t1form,t1p0,t1p1,t1p2,t1p3,t1p4,t1p5,t1p6,t1p7,t1p8,t1p9,t1p10,t1p11,t1p12,t1p13,t2form,t2p0,t2p1,t2p2,t2p3,t2p4,t2p5,t2p6,t2p7,t2p8,t2p9,t2p10,t2p11,t2p12,t2p13,t1Score,t2Score,winner
3-6-1,7.9,9.5,5.3,9.4,6.4,9.8,8.7,8.5,8.2,6.4,8.5,9.4,5.9,6.1,4-3-2-1,5.6,7,8.4,8.8,6.9,6.6,8.8,7.7,8.3,7.5,6.1,5.3,6.1,5.3,2,0,1
4-4-2,5.1,5.7,5.3,9.1,6.9,8.4,7.6,9.8,6.3,9.4,8.7,6.7,8.6,7,4-4-2,6,9.9,7.9,7.2,5.6,7.5,9.1,7.9,9.6,6.6,9.9,8.8,7.8,9.9,0,3,2
3-6-1,7.2,5.2,8.2,6.2,6.5,5.7,8.6,6.1,9.3,7.5,6.6,6.4,7.1,6.9,3-6-1,8.2,9,6.1,9.7,5.2,8,5.2,7.3,5.5,7.4,7.1,5.4,5.1,6.4,2,1,1
4-3-2-1,7.4,7.4,5.2,7.9,5,7.6,8.5,8.9,8.7,7.4,7.6,8.4,7.7,5.5,3-4-3,5.1,9.1,5.8,5.8,9.9,9.1,9.4,7.5,7.6,6.4,9.7,7.9,8.2,5.5,2,0,1


In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

### Pre-processing : one-hot categoty columns

In [9]:
df = pd.read_csv('./data/totalScor_1121.csv')

df = pd.concat([df,pd.get_dummies(df[['t1form']],
                                  prefix='t1form',dummy_na=True)], axis=1).drop(['t1form'],axis=1)
df = pd.concat([df,pd.get_dummies(df[['t2form']],
                                  prefix='t2form',dummy_na=True)], axis=1).drop(['t2form'],axis=1)

df = df.drop(['t2form_nan','t1form_nan'],axis=1)

### Standardization : numerical columns

In [10]:
columns_to_norm = ['t1p0', 't1p1', 't1p2', 't1p3', 't1p4', 't1p5', 't1p6', 't1p7', 't1p8',
                   't1p9', 't1p10', 't1p11', 't1p12', 't1p13', 't2p0', 't2p1', 't2p2',
                   't2p11', 't2p12', 't2p13', 't1Score', 't2Score',
                  ]

In [11]:
df[columns_to_norm] = df[columns_to_norm].apply(lambda x:(x-x.mean()) / x.std())

## Build Model

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import datasets, model_selection
from sklearn.decomposition import PCA

In [14]:
df = df.sample(n=20000, weights='winner', random_state=1)

In [15]:
df.shape

(20000, 53)

In [16]:
y = [1 if x == 1 else 0 for x in df['winner']] 
df = df.drop(['winner'],axis=1)
X = df.values

In [17]:
models = []
#models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('xgboost', XGBClassifier()))

In [18]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3)

In [19]:
predictor = {}
for name, model in models:
    kfold = model_selection.KFold(n_splits=5)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    predictor[name] = model.fit(X_train, y_train)
    print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))

SVM: 1.000000 (0.000000)
xgboost: 1.000000 (0.000000)
